In [ ]:
old_ENV_PATH = ENV["PATH"]
ENV["PATH"] = "/opt/cuda/bin:$old_ENV_PATH"

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/byoungsoo/Notebooks/project_place_cell/figures/output/sfigure11/light_dark"

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_all = [
    ["20230111_112012",1,"byoungsoo"],
    ["20230111_143729",5,"byoungsoo"],
    ["20221223_115022",6,"byoungsoo"]
]

In [ ]:
analyzer = "byoungsoo"

In [ ]:
n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
specificity_diff_tog_1 = []
specificity_diff_tog_2 = []
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_analyzer_1), "compare_map_results_original.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
    
    
    info_filename = joinpath(data_path(ds_save_analyzer_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
    
    figure(figsize=(20,1))
    subplot(1,5,1)
    density_corr_trials, bins = hist(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    xlabel("PF correlation")
    ylabel("density")
    title(experiment_filename_1)
    axvline(nanmedian(corr_trials_all[tel_place_cell_index]), color="r")
    xlim(-1,1)

    subplot(1,5,2)
    density_corr_pv_normalized, bins = hist(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    xlabel("PV correlation normalzied")
    ylabel("density")
    title(experiment_filename_1)
    axvline(nanmedian(corr_pv_all_normalized), color="r")
    xlim(-1,1)

    
    subplot(1,5,3)
    density_shift, bins = hist(peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index)], bins= numpy.linspace(0, 50,n_bins), density=true)
    axvline(nanmedian(peak_shift_mm), color="r")
    xlabel("peak_shift_mm")
    ylabel("density")
    title(experiment_filename_1)
    xlim(0,50)
    
    
    specificity_diff_1 = (specificity_2[tel_place_cell_index_1] .- specificity_1[tel_place_cell_index_1])
    subplot(1,5,4)
    density_specificity_diff_1, bins = hist(specificity_diff_1, bins= numpy.linspace(-0.1, 0.25,n_bins), density=true)
    axvline(nanmedian(specificity_diff_1), color="r")
    xlabel("Δ Specificity")
    ylabel("density")
    title(experiment_filename_1)
    xlim(-0.5, 0.5)
    
    specificity_diff_2 = (specificity_1[tel_place_cell_index_2] .- specificity_2[tel_place_cell_index_2])
    subplot(1,5,5)
    density_specificity_diff_2, bins = hist(specificity_diff_2, bins= numpy.linspace(-0.25, 0.25,n_bins), density=true)
    axvline(nanmedian(specificity_diff_2), color="r")
    xlabel("Δ Specificity")
    ylabel("density")
    title(experiment_filename_1)
    xlim(-0.5, 0.5)
    
    append!(corr_trials_all_tog, [density_corr_trials])
    append!(corr_pv_all_normalized_tog, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog, [density_shift])
    append!(specificity_diff_tog_1, [density_specificity_diff_1])
    append!(specificity_diff_tog_2, [density_specificity_diff_2])
    
end


figure(figsize=(20,1))

subplot(1,5,1)
bins = numpy.linspace(-1,1,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
y = mean(corr_trials_all_tog)
bar(x, y, width=bins[2] - bins[1])
xlabel("PF correlation")
ylabel("density")
tog_mean = sum(x.*y)/sum(y) 
axvline(tog_mean, color="r",label = tog_mean)
xlim(-1,1)
# title(tog_mean)
legend()



subplot(1,5,2)
bins = numpy.linspace(-1,1,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
y = mean(corr_pv_all_normalized_tog)
bar(x, y, width=bins[2] - bins[1])
xlabel("PV correlation normalized")
ylabel("density")
tog_mean = sum(x.*y)/sum(y)
axvline(tog_mean, color="r",label = tog_mean)
xlim(-1,1)
# title(tog_mean)
legend()

subplot(1,5,3)
bins = numpy.linspace(0, 50,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
y = mean(peak_shift_mm_tog)
bar(x, y, width=bins[2] - bins[1])
xlabel("peak_shift_mm_tog")
ylabel("density")
tog_mean = sum(x.*y)/sum(y)
axvline(tog_mean, color="r",label = tog_mean)
xlim(0, 50)
# title(tog_mean)
legend()


subplot(1,5,4)
bins = numpy.linspace(-0.25, 0.25 ,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
y = mean(specificity_diff_tog_1)
bar(x, y, width=bins[2] - bins[1])
xlabel("Δ Specificity")
ylabel("density")
tog_mean = sum(x.*y)/sum(y)
axvline(tog_mean, color="r",label = numpy.round(tog_mean, 3))
xlim(-0.5, 0.5)
# title(tog_mean)
legend()

subplot(1,5,5)
bins = numpy.linspace(-0.25, 0.25 ,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
y = mean(specificity_diff_tog_2)
bar(x, y, width=bins[2] - bins[1])
xlabel("Δ Specificity")
ylabel("density")
tog_mean = sum(x.*y)/sum(y)
axvline(tog_mean, color="r",label = numpy.round(tog_mean, 3))
xlim(-0.5, 0.5)
# title(tog_mean)
legend()

In [ ]:
n_bins = 31

corr_trials_all_tog_control = []
corr_pv_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []
specificity_diff_tog_1_control = []
specificity_diff_tog_2_control = []
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_analyzer_1), "compare_map_results_original.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
    
    
    info_filename = joinpath(data_path(ds_save_analyzer_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1_early"])
    specificity_2 = HDF5.readmmap(file["specificity_1_late"])
    close(file)
    
    figure(figsize=(20,1))
    subplot(1,5,1)
    density_corr_trials, bins = hist(corr_trials_all[tel_place_cell_index], bins= numpy.linspace(-1,1,n_bins), density=true)
    xlabel("PF correlation")
    ylabel("density")
    title(experiment_filename_1)
    axvline(nanmedian(corr_trials_all[tel_place_cell_index]), color="r")
    xlim(-1,1)

    subplot(1,5,2)
    density_corr_pv_normalized, bins = hist(corr_pv_all_normalized, bins= numpy.linspace(-1,1,n_bins), density=true)
    xlabel("PV correlation normalzied")
    ylabel("density")
    title(experiment_filename_1)
    axvline(nanmedian(corr_pv_all_normalized), color="r")
    xlim(-1,1)

    
    subplot(1,5,3)
    density_shift, bins = hist(peak_shift_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index_early_late)], bins= numpy.linspace(0, 50,n_bins), density=true)
    axvline(nanmedian(peak_shift_mm), color="r")
    xlabel("peak_shift_mm")
    ylabel("density")
    title(experiment_filename_1)
    xlim(0,50)
    
    
    specificity_diff_1 = (specificity_2[tel_place_cell_index_1] .- specificity_1[tel_place_cell_index_1])
    subplot(1,5,4)
    density_specificity_diff_1, bins = hist(specificity_diff_1, bins= numpy.linspace(-0.1, 0.25,n_bins), density=true)
    axvline(nanmedian(specificity_diff_1), color="r")
    xlabel("Δ Specificity")
    ylabel("density")
    title(experiment_filename_1)
    xlim(-0.5, 0.5)
    
    specificity_diff_2 = (specificity_1[tel_place_cell_index_2] .- specificity_2[tel_place_cell_index_2])
    subplot(1,5,5)
    density_specificity_diff_2, bins = hist(specificity_diff_2, bins= numpy.linspace(-0.25, 0.25,n_bins), density=true)
    axvline(nanmedian(specificity_diff_2), color="r")
    xlabel("Δ Specificity")
    ylabel("density")
    title(experiment_filename_1)
    xlim(-0.5, 0.5)

    
    append!(corr_trials_all_tog_control, [density_corr_trials])
    append!(corr_pv_all_normalized_tog_control, [density_corr_pv_normalized])
    append!(peak_shift_mm_tog_control, [density_shift])
    append!(specificity_diff_tog_1_control, [density_specificity_diff_1])
    append!(specificity_diff_tog_2_control, [density_specificity_diff_2])
end


In [ ]:
using Printf
function plot_mean_density(bins, y_tog; plot_color="k", shade_color="gray", line_color="k", plot_median = true)
    
    n_bins = length(bins)
    x = (bins[1:end-1] + bins[2:end])/2
    y = mean(y_tog)
    ymax = y + std(y_tog)
    ymin = y - std(y_tog);
    
    cumsum_y_mean = cumsum(mean(y_tog)*(maximum(bins) - minimum(bins))/(n_bins-1))

    x_median = numpy.round(numpy.interp(0.5, cumsum_y_mean, x), 2)
    x_14 = numpy.round(numpy.interp(0.25, cumsum_y_mean, x), 2)
    x_34 = numpy.round(numpy.interp(0.75, cumsum_y_mean, x), 2)
    
    
    plot(x, y, color= plot_color)
    fill_between(x, ymax, ymin, color=shade_color, alpha=0.3)
    if plot_median
        axvline(x_median, color=line_color, linestyle="dashed")
    end
end

In [ ]:
bins = numpy.linspace(0, 50,n_bins)
n_bins = length(bins)
x = (bins[1:end-1] + bins[2:end])/2
y_tog = peak_shift_mm_tog
cumsum_y_mean = cumsum(mean(y_tog)*(maximum(bins) - minimum(bins))/(n_bins-1))

x_median = numpy.round(numpy.interp(0.5, cumsum_y_mean, x), 2)
x_14 = numpy.round(numpy.interp(0.25, cumsum_y_mean, x), 2)
x_34 = numpy.round(numpy.interp(0.75, cumsum_y_mean, x), 2)

println(x_median)
println(x_14)
println(x_34)

In [ ]:
fig = figure(figsize=(0.8, 0.8))
bins = numpy.linspace(-1,1,n_bins)
plot_mean_density(bins, corr_trials_all_tog_control)
plot_mean_density(bins, corr_trials_all_tog;plot_color="r", shade_color="r", line_color="r")
xlabel("PF cor.", labelpad = 0)
ylabel("Density", labelpad = -4)
xlim(-1,1)
legend(frameon=false, loc= (0,1))
yticks([0,3])
fig.savefig(joinpath(fig_path, "PF_correlation.pdf"), bbox_inches="tight",transparent = true);

fig = figure(figsize=(0.8, 0.8))
bins = numpy.linspace(-1,1,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
plot_mean_density(bins, corr_pv_all_normalized_tog_control)
plot_mean_density(bins, corr_pv_all_normalized_tog;plot_color="r", shade_color="r", line_color="r")
xlabel("PV cor.", labelpad = 0)
xlim(-1,1)
legend(frameon=false, loc= (0,1))
# yticks([0,3])
fig.savefig(joinpath(fig_path, "PV_correlation.pdf"), bbox_inches="tight",transparent = true);

fig = figure(figsize=(0.8, 0.8))
bins = numpy.linspace(0, 50,n_bins)
x = (bins[1:end-1] + bins[2:end])/2
plot_mean_density(bins, peak_shift_mm_tog_control)
plot_mean_density(bins, peak_shift_mm_tog;plot_color="r", shade_color="r", line_color="r")
xlabel("PF shift (mm)", labelpad = 0)
xlim(0, 50)
legend(frameon=false, loc= (0,1))
fig.savefig(joinpath(fig_path, "peak_shift.pdf"), bbox_inches="tight",transparent = true);

In [ ]:
n_bins = 31

corr_trials_all_tog = []
corr_pv_all_normalized_tog = []
peak_shift_mm_tog = []
specificity_diff_tog_1 = []
specificity_diff_tog_2 = []
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    info_filename = joinpath(data_path(ds_save_analyzer_1), "compare_map_results_original.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1"])
    specificity_2 = HDF5.readmmap(file["specificity_2"])
    close(file)
    
    
    info_filename = joinpath(data_path(ds_save_analyzer_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)

    append!(corr_trials_all_tog, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_tog, [pv_map_normalized])
    append!(peak_shift_mm_tog, [peak_shift_mm[whether_in(confined_place_cell_index, confined_place_cell_index_early_late)]])
end

In [ ]:
n_bins = 31

corr_trials_all_tog_control = []
corr_pv_all_normalized_tog_control = []
peak_shift_mm_tog_control = []

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    info_filename = joinpath(data_path(ds_save_analyzer_1), "compare_map_results_original.h5")
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    tel_place_cell_index_1 = HDF5.readmmap(file["tel_place_cell_index_1"])
    tel_place_cell_index_2 = HDF5.readmmap(file["tel_place_cell_index_2"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    close(file)
    
    
    info_filename = joinpath(data_path(ds_save_analyzer_1), "compare_map_results_early_late.h5")
    file = h5open(info_filename, "r")
    confined_place_cell_index_early_late = HDF5.readmmap(file["confined_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_earlylate_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_early_late_mm"])
    specificity_1 = HDF5.readmmap(file["specificity_1_early"])
    specificity_2 = HDF5.readmmap(file["specificity_1_late"])
    close(file)

    
    append!(corr_trials_all_tog_control, [corr_trials_all[tel_place_cell_index]])
    append!(corr_pv_all_normalized_tog_control, [pv_map_normalized])
    append!(peak_shift_mm_tog_control, [peak_shift_mm[whether_in(confined_place_cell_index_early_late, confined_place_cell_index)]])
end


In [ ]:
for which_fish = 1:length(corr_trials_all_tog)
x = reduce(vcat, corr_trials_all_tog[which_fish])
y = reduce(vcat, corr_trials_all_tog_control[which_fish])

valid_neuron = isfinite.(x) .* isfinite.(y)
_, p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
println(p_value)
end

In [ ]:
x = reduce(vcat, corr_trials_all_tog)
y = reduce(vcat, corr_trials_all_tog_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")

In [ ]:
for which_fish = 1:length(corr_pv_all_normalized_tog_control)
x = reduce(vcat, corr_pv_all_normalized_tog[which_fish])
y = reduce(vcat, corr_pv_all_normalized_tog_control[which_fish])

valid_neuron = isfinite.(x) .* isfinite.(y)
_, p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")
println(p_value)
end

In [ ]:
x = reduce(vcat, corr_pv_all_normalized_tog)
y = reduce(vcat, corr_pv_all_normalized_tog_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "greater")

In [ ]:
for which_fish = 1:length(peak_shift_mm_tog_control)
x = reduce(vcat, peak_shift_mm_tog[which_fish])
y = reduce(vcat, peak_shift_mm_tog_control[which_fish])

valid_neuron = isfinite.(x) .* isfinite.(y)
_, p_value = stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")
println(p_value)
end

In [ ]:
x = reduce(vcat, peak_shift_mm_tog)
y = reduce(vcat, peak_shift_mm_tog_control)

valid_neuron = isfinite.(x) .* isfinite.(y)
stats.wilcoxon(x[valid_neuron], y[valid_neuron], alternative = "less")